In [80]:
using CSV, DataFrames, JuMP, Gurobi, LinearAlgebra, Random

In [81]:
# Constants

nP = 73 # number of people
nT = 210 # number of timeslots in a week
# Organized in 30 minute interals from 9am-12pm, Sunday-Saturday
# 30 intervals/day * 7 days/week
cM = 1e7 # multiplier for the minimal matching objective

print("")

In [82]:
# Load Data

# 0-1 matrix of whether person can mentor/mentee at time T
load_A = Matrix(CSV.read("schedules.csv", DataFrame, header=0))[3:end, 2:end]  # nP x nT
A = zeros(nP, nT)
for i in 1:nP, j in 1:nT
    A[i, j] = parse(Int64, load_A[i, j])
end

@show size(A)

# for these, what if someone is only a mentor or only a mentee? would it be zero?
aR = Matrix(CSV.read("num_sessions.csv", DataFrame, header=0))[:, 2] # nP x 1: max num of session person can mentor
aE = Matrix(CSV.read("num_sessions.csv", DataFrame, header=0))[:, 3] # nP x 1: max num of sessions person can mentee (0 or 1)
# 
# aE = Matrix(CSV.read("max_menteeing.csv", DataFrame)) # nP x 1: max num of session person can mentee (can we remove this?)
r  = Matrix(CSV.read("rank.csv", DataFrame, header=0))[:, 2]# nP x 1, rank

# Don't we just need one per person? Lessens the amount of data, plus not everyone is a mentor or mentee
p = Matrix(CSV.read("poomsae_pref.csv", DataFrame, header=0))[:, 2]
s = Matrix(CSV.read("sparring_pref.csv", DataFrame, header=0))[:, 2]
# pR = Matrix(CSV.read("mentor_poomsae.csv", DataFrame)) # nP x 1: 0-1 does mentor prefer poomsae 
# pE = Matrix(CSV.read("mentee_poomsae.csv", DataFrame)) # nP x 1: 0-1 does mentee prefer poomsae
# sR = Matrix(CSV.read("mentor_sparring.csv", DataFrame)) # nP x 1: 0-1 does mentor prefer sparring
# sE = Matrix(CSV.read("mentee_sparring.csv")) # nP x 1: 0-1 does mentee prefer sparring

#@show aR
@show aE
#@show r
#@show p
#@show s


print("")

size(A) = (73, 210)
aE = Any[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [83]:
# Create Cost Matrix

alpha = 1
beta  = 1
gamma = 1

C = zeros(nP, nP)

# Adjusting to only have one set of data for sparring and poomsae preferences
for i = 1:nP, j = 1:nP
    C[i, j] += alpha * (A[i]'*A[j])
    C[i, j] += beta * (r[j]-r[i]-2+(r[j] <= 0))
    C[i, j] -= gamma  * (p[i]*p[j]+s[i]*s[j]-p[i]*s[i]-p[j]*s[j]))
    C[i, j] -= 1
end

# for i = 1:nP, j = 1:nP
#     C[i, j] += alpha * (r[i]-r[j]-2+(r[j] >= 9))
#     C[i, j] -= beta  * (pR[i]*pE[j]+sR[i]*sE[j]-pR[i]*sR[i]-pE[j]*sE[j])
#     C[i, j] -= 1
# end


LoadError: syntax: "for" at In[83]:10 expected "end", got ")"

In [84]:
# Rank Constraint Matrix
# Adjust to be negative hehe

R = zeros(nP, nP)

for i = 1:nP, j = 1:nP
    if r[i] <= r[j]-2 || (r[i] <= r[j]-1 && r[j] <= 0)
        R[i, j] = 1
    end
end

R = convert(Array{Int64}, R);

In [85]:
m = Model(Gurobi.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)
set_optimizer_attribute(m, "IntegralityFocus",1)

print("")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-03
Set parameter IntegralityFocus to value 1


In [86]:
@variable(m, 0 <= x[1:nP, 1:nP] <= 1)
@variable(m, 0 <= t[1:nP] <= 1, Int)

print("")

In [87]:
# Rank Constraint (capacity constraint)
@constraint(m, x .<= R);

In [88]:
# Maximum Constraints
for P = 1:nP
    @constraint(m, sum(x[i, P] for i=1:nP) <= aE[P]);
    @constraint(m, sum(x[P, i] for i=1:nP) <= aR[P]); # replaced with ones
end

In [89]:
# # Availability Constraints
# for i = 1:nP, j = i+1:nP, T=1:nT
#     @constraint(m, x[i,j,T] <= A[i,T]*A[j,T]);
# end

In [90]:
# # THIS ONE TAKES TOO LONG

# # Single Mentor Constraint
# for i = 1:nP, j = i+1:nP, k = 1:nP, T=1:nT
#     @constraint(m, x[i,k,T]+x[j,k,T] <= 1);
# end

In [91]:
# Making t work
for P = 1:nP
    @constraint(m, sum(x[i,P] for i=1:nP) >= t[P])
    @constraint(m, 0.01*sum(x[i,P] for i=1:nP) <= t[P]);  # make constraint integral
end

print("")

In [ ]:
@objective(m, Min, cM*sum((1-t[P])*aE[P] for P=1:nP) + sum(x[i,j]*C[i,j] for i=1:nP, j=1:nP))
print("")

In [ ]:
optimize!(m)

objective_value(m)

In [ ]:
count = 0
x_sol = value.(x)
for i in 1:nP, j in 1:nP
    if x_sol[i, j] != 0
        count += 1
    end
end
@show count

In [ ]:
names = Matrix(CSV.read("num_sessions.csv", DataFrame, header=0))[:, 1]
# @show names

# count = 0
# x_sol = value.(x)
for i in 1:nP, j in 1:nP
    if x_sol[i, j] != 0
        print("$(names[i]), $(names[j]), $(x_sol[i, j])\n")
    end
end
# @show count

In [ ]:
```
Analysis:
-Generate an x representing the existing mentorship pairs and calculate the cost
-Report how many people who wanted a mentor received one
-

```

In [ ]:
baseline_x_load = Matrix(CSV.read("baseline.csv", DataFrame, header=0))[2:end, 2:end]
baseline_x = zeros(nP, nP)
for i in 1:nP, j in 1:nP
    baseline_x[i, j] = parse(Int64, baseline_x_load[i, j])
end
baseline_x = convert(Matrix{Int64}, baseline_x)

In [ ]:
baseline_x
baseline_t = sum(baseline_x, dims=1)

baseline_cost = cM*sum((1-baseline_t[P])*aE[P] for P=1:nP) + sum(baseline_x[i,j]*C[i,j] for i=1:nP, j=1:nP)